<a href="https://colab.research.google.com/github/ReynaQuita/NLP/blob/main/Translation_t5_fine_tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers sacrebleu

     |████████████████████████████████| 225kB 3.9MB/s 
     |████████████████████████████████| 2.3MB 17.3MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 245kB 35.8MB/s 
     |████████████████████████████████| 112kB 28.5MB/s 
     |████████████████████████████████| 901kB 28.9MB/s 
     |████████████████████████████████| 3.3MB 32.9MB/s 
ERROR: transformers 4.6.1 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.9 which is incompatible.


If you're opening this notebook locally, make sure your environment has the last version of those libraries installed.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](https://github.com/huggingface/notebooks/blob/master/examples/images/translation.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [ ]:
# model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"
# model_checkpoint = "bert-base-uncased"
# model_checkpoint = "gpt2"
model_checkpoint = "t5-small"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [ ]:
from datasets import load_dataset, load_metric

train_datasets = load_dataset("wmt16", "ro-en", split="train[:1%]")
val_datasets = load_dataset("wmt16", "ro-en", split="validation[:5%]")
metric = load_metric("sacrebleu")

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


Reusing dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
train_datasets

Dataset({
    features: ['translation'],
    num_rows: 6103
})

In [ ]:
val_datasets

Dataset({
    features: ['translation'],
    num_rows: 100
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
train_datasets[0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_datasets)

,translation
0,"{'en': 'I am basically referring to the principles whereby the Member States should define and delimit the supply and content of the public service licences taken up by the corresponding public bodies, and should have the freedom to choose the funding model, thereby avoiding any distortions to free competition.', 'ro': 'Mă refer, esenţialmente, la principiile pe baza cărora statele membre trebuie să-şi definească şi să-şi delimiteze oferta şi conţinutul licenţelor de utilitate publică pentru organele publice corespunzătoare, având totodată dreptul să-şi selecteze propriul model de finanţare, evitând în acest fel denaturarea liberei concurenţe.'}"
1,"{'en': 'Agenda for next sitting: see Minutes', 'ro': 'Ordinea de zi a următoarei şedinţe: a se vedea procesul-verbal'}"
2,"{'en': '(LT) I would also like to underline our rapporteur's excellent work and ability to find a suitable compromise.', 'ro': '(LT) Aş dori să subliniez şi eu activitatea excelentă a domnului raportor şi capacitatea sa de a ajunge la un compromis convenabil.'}"
3,"{'en': 'Written declarations for entry in the register (Rule 116): see Minutes', 'ro': 'Declaraţii scrise înscrise în registru (articolul 116 din Regulamentul de procedură): a se vedea procesul-verbal'}"
4,"{'en': 'rapporteur. - (DE) Madam President, ladies and gentlemen, it is now possible to identify a sugar cube in Lake Constance using state-of-the-art analysis.', 'ro': 'raportoare. -(DE) Doamnă preşedintă, doamnelor şi domnilor, la ora actuală este posibilă identificarea unui cub de zahăr în Lacul Constance folosind o analiză ultramodernă.'}"


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments)
    references: A list of one or more reference streams (each a sequence of segments)
    smooth: The smoothing method to use
    smooth_value: For 'floor' smoothing, the floor to use
    force: Ignore data that looks already tokenized
    lowercase: Lowercase the data
    tokenize: The tokenizer to use
Returns:
    'score': BLEU score,
    'counts': Counts,
    'totals': Totals,
    'precisions': Precisions,
    'bp': Brevity penalty,
    'sys_len': predictions length,
    'ref_len': reference length,
Examples:

    >>> predictions = ["hello there general kenobi", "foo bar foobar"]
    >>> references = [["hello there gen

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(train_datasets[:2])

{'input_ids': [[13959, 1566, 12, 3871, 29, 10, 19428, 13, 12876, 10, 217, 13687, 7, 1], [13959, 1566, 12, 3871, 29, 10, 2276, 8843, 138, 13, 13687, 7, 13, 1767, 3823, 10, 217, 13687, 7, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[4961, 106, 20437, 13636, 252, 10, 3, 9, 142, 11002, 15948, 18, 11868, 138, 1], [5085, 5840, 498, 6345, 252, 18, 11868, 138, 491, 28109, 15, 23, 17875, 15, 10, 3, 9, 142, 11002, 15948, 18, 11868, 138, 1]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_train_datasets = train_datasets.map(preprocess_function, batched=True)

In [ ]:
tokenized_val_datasets = val_datasets.map(preprocess_function, batched=True)

In [ ]:
len(tokenized_train_datasets)

6103

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 16
# args = Seq2SeqTrainingArguments(
#     "test-translation",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True,
#     fp16=True,
# )
args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=10,
    eval_steps=64,
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu,Gen Len
64,0.621700,1.355220,11.010900,17.970000
128,0.605100,1.371934,10.947800,17.950000
192,0.521200,1.376304,10.957400,17.970000
256,0.901900,1.378288,11.023100,17.980000
320,0.644600,1.383759,11.407000,17.960000
384,0.624500,1.384234,11.363100,17.960000
448,0.806700,1.389713,11.309600,17.980000
512,0.688600,1.394317,11.212200,17.980000
576,0.587400,1.395648,11.335000,17.980000
640,0.641700,1.397683,11.222100,18.000000


TrainOutput(global_step=1146, training_loss=0.6139556238684563, metrics={'train_runtime': 1022.1211, 'train_samples_per_second': 1.121, 'total_flos': 4681034459136.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1821495296, 'init_mem_gpu_alloc_delta': 242026496, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -246034432, 'train_mem_gpu_alloc_delta': 732158464, 'train_mem_cpu_peaked_delta': 263892992, 'train_mem_gpu_peaked_delta': 2176835072})

Don't forget to [upload your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!

In [ ]:
!ls

checkpoint-10	 checkpoint-180  checkpoint-40	 checkpoint-610  checkpoint-830
checkpoint-100	 checkpoint-190  checkpoint-400  checkpoint-620  checkpoint-840
checkpoint-1000  checkpoint-20	 checkpoint-410  checkpoint-630  checkpoint-850
checkpoint-1010  checkpoint-200  checkpoint-420  checkpoint-640  checkpoint-860
checkpoint-1020  checkpoint-210  checkpoint-430  checkpoint-650  checkpoint-870
checkpoint-1030  checkpoint-220  checkpoint-440  checkpoint-660  checkpoint-880
checkpoint-1040  checkpoint-230  checkpoint-450  checkpoint-670  checkpoint-890
checkpoint-1050  checkpoint-240  checkpoint-460  checkpoint-680  checkpoint-90
checkpoint-1060  checkpoint-250  checkpoint-470  checkpoint-690  checkpoint-900
checkpoint-1070  checkpoint-260  checkpoint-480  checkpoint-70	 checkpoint-910
checkpoint-1080  checkpoint-270  checkpoint-490  checkpoint-700  checkpoint-920
checkpoint-1090  checkpoint-280  checkpoint-50	 checkpoint-710  checkpoint-930
checkpoint-110	 checkpoint-290  checkpoint-500  

# **EVALUATION IN TEST DATASET**

In [ ]:
model_fine_tuned = AutoModelForSeq2SeqLM.from_pretrained("./checkpoint-1140")

In [ ]:
tokenizer_fine_tuned = AutoTokenizer.from_pretrained("./checkpoint-1140")

In [ ]:
test_data = datasets.load_dataset("wmt16", "ro-en", split="test[:5%]")
len(test_data)

Reusing dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)


100

In [ ]:
def generate_translation(batch):
    # cut off at BERT max length 512

    english = [ex["en"] for ex in batch["translation"]]
    # romanian = [ex["ro"] for ex in batch["translation"]]

    inputs = tokenizer(english, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    # input_ids = inputs.input_ids.to("cuda")
    # attention_mask = inputs.attention_mask.to("cuda")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model_fine_tuned.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer_fine_tuned.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_translation"] = output_str

    return batch

In [ ]:
batch_size = 4  # change to 64 for full evaluation

results = test_data.map(generate_translation, batched=True, batch_size=batch_size)

In [ ]:
test_data["translation"][0]

{'en': 'UN Chief Says There Is No Military Solution in Syria',
 'ro': 'Șeful ONU declară că nu există soluții militare în Siria'}

In [ ]:
ref = results["translation"][0]["ro"]
print(ref)

Șeful ONU declară că nu există soluții militare în Siria


In [ ]:
pred = results["pred_translation"][0]
print(pred)

Chef des Nations Unies a déclaré qu'il n'y a aucune


In [ ]:
metric.compute(predictions=[pred], references=[[ref]])

{'bp': 1.0,
 'counts': [0, 0, 0, 0],
 'precisions': [5.0, 2.7777777777777777, 1.5625, 0.8928571428571429],
 'ref_len': 10,
 'score': 2.0980574531482743,
 'sys_len': 10,
 'totals': [10, 9, 8, 7]}

In [ ]:
bleu_score = 0

for i in range(len(results)):
  ref = results["translation"][i]["ro"]
  pred = results["pred_translation"][i]

  bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
  bleu_score  += bleu_metric["score"]


In [ ]:
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 3.0683811853424174 (scale 0-100)
